In [29]:
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph.message import add_messages
from langgraph.graph import StateGraph, START, END
from langchain_google_genai import ChatGoogleGenerativeAI
import os

In [30]:
# Configure LLM
apikey = os.getenv('GOOGLE_API_KEY')
llm = ChatGoogleGenerativeAI(model='gemini-2.5-flash-lite',api_key=apikey)

In [31]:
class Chatbot_State(TypedDict):
    message : Annotated[list,add_messages]

In [32]:
def chatbot(state:Chatbot_State) -> Chatbot_State:
    return {'message':[llm.invoke(state['message'])]}

In [33]:
# LangGraph

builder = StateGraph(Chatbot_State)

builder.add_node('chatting_bot',chatbot)

builder.add_edge(START,"chatting_bot")
builder.add_edge('chatting_bot',END)

graph = builder.compile()

In [34]:
print(graph.get_graph().draw_ascii())

  +-----------+  
  | __start__ |  
  +-----------+  
        *        
        *        
        *        
+--------------+ 
| chatting_bot | 
+--------------+ 
        *        
        *        
        *        
  +---------+    
  | __end__ |    
  +---------+    


In [35]:
response = graph.invoke({'message':{'role':'user','content':'Who is MS Dhoni?'}})
print(response['message'][-1].content)

MS Dhoni is a legendary **Indian former international cricketer** who is widely regarded as one of the greatest captains, wicket-keeper batsmen, and finishers in the history of the sport.

Here are some key aspects of his career and legacy:

*   **Captain Fantastic:** He is most famous for his incredibly successful captaincy of the Indian cricket team across all three formats (Test, One Day International (ODI), and Twenty20 International (T20I)). Under his leadership, India achieved unprecedented success, including:
    *   **ICC T20 World Cup 2007:** A historic victory in the inaugural T20 World Cup, which marked a turning point for Indian cricket and the T20 format.
    *   **ICC Cricket World Cup 2011:** Leading India to their second ODI World Cup title after a 28-year drought, a moment of immense national pride.
    *   **ICC Champions Trophy 2013:** Making India the only nation to win all three major ICC limited-overs tournaments.
    *   **ICC Test Mace:** India also reached the 

In [36]:
# CHATBOT

while True:
  user_input=input()
  print('User:',user_input)
  if user_input.lower() in ["quit","q"]:
    print("Good Bye")
    break
  for event in graph.stream({'message':("user",user_input)}):
    for value in event.values():
      print("Assistant:",value["message"][-1].content)

User: hi
Assistant: Hi there! How can I help you today?
User: What is the capital of france?
Assistant: The capital of France is **Paris**.
User: What the date today?
Assistant: I am a large language model, I do not have access to real-time information including the current date.
User: q
Good Bye
